In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras import layers


Num GPUs Available:  1


In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
DATASET_PATH = '/home/datasets/Project_data/'

## Data Generator

In [4]:
class DataGenerator():
    def __init__(self):
        'Initialization'
        self.train_doc = np.random.permutation(open(DATASET_PATH + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(DATASET_PATH + 'val.csv').readlines())
        
        self.train_path = DATASET_PATH + 'train'
        self.val_path = DATASET_PATH + 'val'
        
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
        self.img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,26,27,28,29]
    
    def initialize_parameters(self,batch_size=10,img_height=100,img_width=100,num_epochs=5,
                              n_channels=3,ablation=None,mode='train'):
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.img_height = img_height
        self.img_width = img_width
        self.channels = n_channels
        self.num_classes = 5
        self.frames = 30
    
    def generator(self,source_path, folder_list, transform=False):
        batch_size=self.batch_size
        
        while True:
            t = np.random.permutation(folder_list)
            num_batches = np.floor(len(folder_list)/self.batch_size).astype(int)
            
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data, batch_labels = self.fetch_batch(source_path,t,batch,self.batch_size,self.img_idx,transform)
                yield batch_data, batch_labels
            
            pending_batches = (len(folder_list) - num_batches*self.batch_size)
            
            if pending_batches > 0:
                batch_data, batch_labels = self.fetch_batch(source_path,t,batch,pending_batches,self.img_idx,transform)
                yield batch_data, batch_labels
    
    def fetch_batch(self,source_path,t,batch,batch_size,img_idx,transform):
        # x is the number of images you use for each video, 
        # (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_data = np.zeros((batch_size,len(img_idx),self.img_height,self.img_width,3))
        batch_labels = np.zeros((batch_size,5))
        
        if (transform):
            batch_data_affine = np.zeros((batch_size,len(img_idx),self.img_height,self.img_width,3))

        for folder in range(batch_size):
            # read all the images in the folder
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
            
            img_idx = self.img_idx
            
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                if image.shape[0] != image.shape[1]:
                    image = image[0:120, 10:150]
                
                image = cv2.resize(image, (self.img_height,self.img_width), interpolation = cv2.INTER_AREA)

                image = image/255
                
                batch_data[folder,idx,:,:,0] = image[:,:,0]
                batch_data[folder,idx,:,:,1] = image[:,:,1]
                batch_data[folder,idx,:,:,2] = image[:,:,2]
                
                #Affine transformation to shift image up/down or right/left
                if (transform):
                    height, width = image.shape[:2]
                    tx, ty = width / 4, height / 4
                    tx = np.random.randint(-tx,tx)
                    ty = np.random.randint(-ty,ty)
                    translation_matrix = np.array([[1, 0, tx],
                                                   [0, 1, ty]], dtype=np.float32)

                    translated_image = cv2.warpAffine(src=image, M=translation_matrix, dsize=(width, height))

                    batch_data_affine[folder,idx,:,:,0] = translated_image[:,:,0]
                    batch_data_affine[folder,idx,:,:,1] = translated_image[:,:,1]
                    batch_data_affine[folder,idx,:,:,2] = translated_image[:,:,2]

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

        if (transform):
            batch_data=np.concatenate([batch_data,batch_data_affine])
            batch_labels=np.concatenate([batch_labels,batch_labels])
        
        return (batch_data, batch_labels)
    
    def train_model(self,model,transform=False):
        train_generator = self.generator(self.train_path, self.train_doc,transform)
        val_generator = self.generator(self.val_path, self.val_doc,transform)
        
        curr_dt_time = datetime.datetime.now()
        
        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto',save_freq = 'epoch')
        
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1) # write the REducelronplateau code here
        
        earlystop = EarlyStopping(monitor="val_loss", min_delta=0,patience=10,verbose=1)
        
        callbacks_list = [checkpoint, LR, earlystop]
        
        if (self.num_train_sequences%self.batch_size) == 0:
            #steps_per_epoch = np.floor(num_train_sequences/batch_size).astype(int)
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            #steps_per_epoch = (num_train_sequences//batch_size) + 1
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            #validation_steps = np.floor(num_val_sequences/batch_size).astype(int)
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            #validation_steps = (num_val_sequences//batch_size) + 1
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
        
        
        history = model.fit(train_generator, 
                            steps_per_epoch=steps_per_epoch, 
                            epochs=self.num_epochs, 
                            verbose=1, 
                            callbacks=callbacks_list, 
                            validation_data=val_generator, 
                            validation_steps=validation_steps, 
                            class_weight=None, workers=1, initial_epoch=0)
        
        return history
    
    def define_model(self):
        pass

## Building Convolution 3D Model

In [8]:
class Conv3DModel5(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_last', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [49]:
conv_3d8_1=Conv3DModel5()
conv_3d8_1.initialize_parameters(batch_size=20,num_epochs=35,img_height=150,img_width=150)
conv_3d8_1_model=conv_3d8_1.model_structure()
conv_3d8_1_model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 16, 148, 148, 16)  1312      
_________________________________________________________________
batch_normalization_61 (Batc (None, 16, 148, 148, 16)  64        
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 8, 74, 74, 16)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 6, 72, 72, 32)     13856     
_________________________________________________________________
batch_normalization_62 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 124416)          

In [50]:
conv_3d8_1.train_model(model=conv_3d8_1_model)

Epoch 1/35
34/34 [==============================] - ETA: 0s - loss: 1.4797 - categorical_accuracy: 0.4676
Epoch 00001: val_loss improved from inf to 2.00414, saving model to model_init_2021-07-0321_40_10.421795/model-00001-1.47974-0.46757-2.00414-0.19000.h5
34/34 [==============================] - 38s 1s/step - loss: 1.4797 - categorical_accuracy: 0.4676 - val_loss: 2.0041 - val_categorical_accuracy: 0.1900
Epoch 2/35
33/34 [============================>.] - ETA: 0s - loss: 1.0207 - categorical_accuracy: 0.6348
Epoch 00002: val_loss did not improve from 2.00414
34/34 [==============================] - 38s 1s/step - loss: 1.0191 - categorical_accuracy: 0.6350 - val_loss: 4.2005 - val_categorical_accuracy: 0.2200
Epoch 3/35
33/34 [============================>.] - ETA: 0s - loss: 0.7144 - categorical_accuracy: 0.7379
Epoch 00003: val_loss did not improve from 2.00414
34/34 [==============================] - 38s 1s/step - loss: 0.7183 - categorical_accuracy: 0.7360 - val_loss: 10.5311 - v

## Building Convolution 2D + RNN Model

In [56]:
class Conv2DModel5(DataGenerator):

    def model_structure(self):
        
        model = Sequential()

        model.add(TimeDistributed(layers.Conv2D(16,(3,3), activation="relu"), 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(TimeDistributed(BatchNormalization()))
        #model.add(layers.Dropout(0.5))
        model.add(TimeDistributed(layers.MaxPool2D()))

        model.add(TimeDistributed(layers.Conv2D(32,(3,3), activation="relu")))
        model.add(TimeDistributed(BatchNormalization()))
        #model.add(layers.Dropout(0.5))
        model.add(TimeDistributed(layers.MaxPool2D()))
        
        model.add(TimeDistributed(layers.Conv2D(64,(3,3), activation="relu")))
        model.add(TimeDistributed(BatchNormalization()))
        #model.add(layers.Dropout(0.5))
        model.add(TimeDistributed(layers.MaxPool2D()))

        model.add(TimeDistributed(layers.Flatten()))
        
        model.add(layers.Dense(64, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))
        
        model.add(layers.Dense(16, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))
        
        model.add(layers.GRU(16, return_sequences=False))
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.01)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

In [57]:
conv_2d5=Conv2DModel5()
conv_2d5.initialize_parameters(batch_size=20,num_epochs=45,img_height=100,img_width=100)
conv_2d5_model=conv_2d5.model_structure()
conv_2d5_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_127 (TimeDi (None, 18, 98, 98, 16)    448       
_________________________________________________________________
time_distributed_128 (TimeDi (None, 18, 98, 98, 16)    64        
_________________________________________________________________
time_distributed_129 (TimeDi (None, 18, 49, 49, 16)    0         
_________________________________________________________________
time_distributed_130 (TimeDi (None, 18, 47, 47, 32)    4640      
_________________________________________________________________
time_distributed_131 (TimeDi (None, 18, 47, 47, 32)    128       
_________________________________________________________________
time_distributed_132 (TimeDi (None, 18, 23, 23, 32)    0         
_________________________________________________________________
time_distributed_133 (TimeDi (None, 18, 21, 21, 64)  

In [58]:
conv_2d5.train_model(model=conv_2d5_model)

Epoch 1/45
33/34 [============================>.] - ETA: 0s - loss: 1.5808 - categorical_accuracy: 0.2879
Epoch 00001: val_loss improved from inf to 2.55047, saving model to model_init_2021-07-0402_44_48.485766/model-00001-1.58128-0.28808-2.55047-0.24000.h5
34/34 [==============================] - 33s 982ms/step - loss: 1.5813 - categorical_accuracy: 0.2881 - val_loss: 2.5505 - val_categorical_accuracy: 0.2400
Epoch 2/45
33/34 [============================>.] - ETA: 0s - loss: 1.3618 - categorical_accuracy: 0.4333
Epoch 00002: val_loss did not improve from 2.55047
34/34 [==============================] - 34s 992ms/step - loss: 1.3607 - categorical_accuracy: 0.4329 - val_loss: 2.5620 - val_categorical_accuracy: 0.2700
Epoch 3/45
34/34 [==============================] - ETA: 0s - loss: 1.1182 - categorical_accuracy: 0.5279
Epoch 00003: val_loss improved from 2.55047 to 2.47476, saving model to model_init_2021-07-0402_44_48.485766/model-00003-1.11817-0.52790-2.47476-0.27000.h5
34/34 [====